In [3]:
from StartingCandidateGenerator import StartingCandidateGenerator as SCG
from DataBaseInterface import DataBaseInterface as DBI
from CrossOperation import CrossOperation as CO
"Import necessary to fix atoms in the slab"
from ase.constraints import FixAtoms
from ase import Atoms
from ase.io import read
from generate_ceo2 import create_ceo2_111 as ceo2Slab
import numpy as np
from ase.visualize import view
import numpy as np
from ase.optimize import BFGS
from ase.calculators.emt import EMT

#--------------------------Define the fitness fucntion for your atoms-----------------------"

def fitness_function(atoms,env,reference = 0.0,au_energy = 0.0)-> float:

    at_num= atoms.get_atomic_numbers()
    pt_num = np.count_nonzero(at_num == 78)
    au_num= np.count_nonzero(at_num == 79)

    fre = atoms.get_potential_energy() - reference - au_num * (au_energy) - au_num*env
    return fre

#---------------------------Generate static part of the system------------------------------"
slab = ceo2Slab(number_of_layers=1,repetitions = (5,5,1),vacuum=12.0)
a = 12.0
slab = Atoms(cell=[a,a,a],
             pbc=True)
c = FixAtoms(indices=np.arange(len(slab)))
slab.set_constraint(c)
#If slab atoms arent fixed they will be generated  for all structures, but will be relaxed!"

#---------------------------Generate constant part of the system------------------------------"
#Part of the system that can be relaxed but that does not vary in number over the duration of the search"
constant = Atoms('Pt3')

#---------Generate variable part of the system-(Single atom types only)------------------"
#Part of the system that can be relaxed and that varyies in number over the duration of the search"
#
variable_type = Atoms('Au')
#Considered number of variable type atoms n the search"
variable_range = [0,1,2,3,4,5,7,8,9,10]

#---------------------------Define starting population--------------------------------"

candidateGenerator = SCG(slab,constant,variable_type,variable_range)
crossing = CO(slab,constant,variable_type,variable_range,stc_change_chance = 0.5)
db = DBI('databaseGA.db')
population = 25

starting_pop = candidateGenerator.get_starting_population(population_size=population)

#-----------------------------------Define Calculator----------------------------------------"

calc = EMT()

#Get Reference energies##########
constant.cell = slab.get_cell()
variable_type.cell = slab.get_cell()

constant.calc = calc
dyn = BFGS(constant)
dyn.run(steps=100, fmax=0.05)
ref = constant.get_potential_energy()
variable_type.calc = calc
dyn = BFGS(variable_type)
dyn.run(steps=100, fmax=0.05)
au_en = variable_type.get_potential_energy()
#####################################



for i in starting_pop:
    db.add_unrelaxed_candidate(i )

#---------------------------------Relax initial structures-----------------------------------"
env = -0.5 #Environment chem pot 
while db.get_number_of_unrelaxed_candidates() > 0:

    atoms = db.get_first_unrelaxed()
    
    atoms.calc = calc
    dyn = BFGS(atoms)
    dyn.run(steps=100, fmax=0.05)
    atoms.get_potential_energy()
    
    atoms.info['key_value_pairs']['raw_score'] = -fitness_function(atoms,env,reference = ref, au_energy = au_en)
   
    db.update_to_relaxed(atoms.info['key_value_pairs']['dbid'],atoms)

#--------------------------------------Find better stoich to srtart eval----------------------------
#Overall fitness value
steps = 100
sub_steps = 20
for i in range(steps):
    for j in range(sub_steps):
        atomslist = db.get_better_candidates(n=10)
        #Choose two of the most stable structures to pair
        cand1 = np.random.randint(0,10)
        cand2 = np.random.randint(0,10)
        while cand1 == cand2:
            cand2 = np.random.randint(0,10)
            
        #Mate the particles
        res  = crossing.cross(atomslist[cand1],atomslist[cand2])
        if(res is not None):
            if(len(atomslist[cand1])!= len(res) and len(atomslist[cand2]) != len(res)):
                print("--------------STCCHANGE-----------------")
            db.add_unrelaxed_candidate(res)
            
    
    while db.get_number_of_unrelaxed_candidates() > 0:

        atoms = db.get_first_unrelaxed()

        atoms.calc = calc
        dyn = BFGS(atoms)
        dyn.run(steps=100, fmax=0.05)
        atoms.get_potential_energy()

        atoms.info['key_value_pairs']['raw_score'] = -fitness_function(atoms,env,reference = ref, au_energy = au_en)

        db.update_to_relaxed(atoms.info['key_value_pairs']['dbid'],atoms)
        
atomslist = db.get_better_candidates(n=2)
for at in atomslist:
    print(at.numbers)
view(atomslist[0])

      Step     Time          Energy         fmax
BFGS:    0 13:28:25    21203.444554           nan
BFGS:    1 13:28:25       17.550000        0.0000
      Step     Time          Energy         fmax
BFGS:    0 13:28:25        3.800000        0.0000
      Step     Time          Energy         fmax
BFGS:    0 13:28:26       15.186556        3.1108
BFGS:    1 13:28:26       14.900050        3.4648
BFGS:    2 13:28:26       13.190140        5.3819
BFGS:    3 13:28:26       10.693512        7.3426
BFGS:    4 13:28:26        7.749519        6.7556
BFGS:    5 13:28:26        6.641222        7.9035
BFGS:    6 13:28:26        6.047287        4.0316
BFGS:    7 13:28:26        5.105131        3.2969
BFGS:    8 13:28:26        5.685408       10.6590
BFGS:    9 13:28:26        4.873915        2.1312
BFGS:   10 13:28:26        4.799954        1.2184
BFGS:   11 13:28:26        4.771322        0.7469
BFGS:   12 13:28:26        4.763894        0.4406
BFGS:   13 13:28:26        4.736777        0.6780
BFG

BFGS:   62 13:28:26        5.434796        0.7526
BFGS:   63 13:28:26        5.433955        0.7540
BFGS:   64 13:28:26        5.431951        0.7731
BFGS:   65 13:28:26        5.426090        0.8949
BFGS:   66 13:28:26        5.414573        1.3601
BFGS:   67 13:28:26        5.406964        1.7086
BFGS:   68 13:28:26        5.397224        1.3535
BFGS:   69 13:28:26        5.384686        1.6591
BFGS:   70 13:28:26        5.366817        2.0892
BFGS:   71 13:28:26        5.348042        2.1346
BFGS:   72 13:28:26        5.317021        1.9307
BFGS:   73 13:28:26        5.291764        1.1748
BFGS:   74 13:28:26        5.175632        1.1756
BFGS:   75 13:28:26        5.129665        1.6451
BFGS:   76 13:28:26        5.074121        1.1490
BFGS:   77 13:28:26        5.073684        1.6587
BFGS:   78 13:28:26        5.040193        1.1901
BFGS:   79 13:28:26        5.023171        0.6999
BFGS:   80 13:28:26        5.012698        0.7993
BFGS:   81 13:28:26        4.998187        0.8261


BFGS:   36 13:28:26        8.400755        0.6658
BFGS:   37 13:28:26        8.385697        0.5668
BFGS:   38 13:28:26        8.377244        0.5043
BFGS:   39 13:28:26        8.344586        0.5274
BFGS:   40 13:28:26        8.315684        0.5020
BFGS:   41 13:28:26        8.300046        0.4682
BFGS:   42 13:28:26        8.291076        0.2568
BFGS:   43 13:28:26        8.287413        0.2125
BFGS:   44 13:28:26        8.285131        0.1983
BFGS:   45 13:28:26        8.283832        0.2148
BFGS:   46 13:28:26        8.282973        0.2116
BFGS:   47 13:28:26        8.281820        0.1743
BFGS:   48 13:28:26        8.280234        0.1205
BFGS:   49 13:28:27        8.278644        0.1109
BFGS:   50 13:28:27        8.277955        0.0882
BFGS:   51 13:28:27        8.277786        0.0794
BFGS:   52 13:28:27        8.277716        0.0714
BFGS:   53 13:28:27        8.277629        0.0683
BFGS:   54 13:28:27        8.277480        0.0838
BFGS:   55 13:28:27        8.277043        0.1144


BFGS:   45 13:28:27        9.317391        0.1328
BFGS:   46 13:28:27        9.315737        0.1870
BFGS:   47 13:28:27        9.312862        0.2374
BFGS:   48 13:28:27        9.305952        0.3051
BFGS:   49 13:28:27        9.286295        0.4083
BFGS:   50 13:28:27        9.225107        0.6846
BFGS:   51 13:28:27        9.065033        1.0108
BFGS:   52 13:28:27        8.790391        1.0006
BFGS:   53 13:28:27        8.814353        2.9240
BFGS:   54 13:28:27        8.693306        0.7700
BFGS:   55 13:28:27        8.660345        0.7809
BFGS:   56 13:28:27        8.629025        0.9994
BFGS:   57 13:28:27        8.605750        0.7835
BFGS:   58 13:28:27        8.519247        0.9275
BFGS:   59 13:28:27        8.501178        0.9862
BFGS:   60 13:28:27        8.466804        0.6857
BFGS:   61 13:28:27        8.458592        0.6702
BFGS:   62 13:28:27        8.448149        0.6207
BFGS:   63 13:28:27        8.438426        0.5638
BFGS:   64 13:28:27        8.382817        0.6210


BFGS:   13 13:28:28       11.915539        0.9591
BFGS:   14 13:28:28       11.745694        1.9098
BFGS:   15 13:28:28       11.651198        1.6313
BFGS:   16 13:28:28       11.454028        1.2327
BFGS:   17 13:28:28       11.331141        1.1086
BFGS:   18 13:28:28       11.132776        1.1136
BFGS:   19 13:28:28       11.061417        1.7650
BFGS:   20 13:28:28       10.951739        1.2358
BFGS:   21 13:28:28       10.794957        0.9179
BFGS:   22 13:28:28       10.788099        1.7621
BFGS:   23 13:28:28       10.675561        1.2479
BFGS:   24 13:28:28       10.623080        0.8028
BFGS:   25 13:28:28       10.543519        0.6910
BFGS:   26 13:28:28       10.464844        1.2772
BFGS:   27 13:28:28       10.402931        0.9603
BFGS:   28 13:28:28       10.347286        0.5921
BFGS:   29 13:28:28       10.299690        0.4029
BFGS:   30 13:28:28       10.285831        0.3932
BFGS:   31 13:28:28       10.261753        0.4655
BFGS:   32 13:28:28       10.240014        0.5308


...

BFGS:   25 14:06:44       10.570835        0.0903
BFGS:   26 14:06:44       10.569601        0.1054
BFGS:   27 14:06:44       10.567754        0.1252
BFGS:   28 14:06:44       10.564043        0.2093
BFGS:   29 14:06:44       10.555444        0.3807
BFGS:   30 14:06:44       10.551679        0.7847
BFGS:   31 14:06:44       10.535066        0.6710
BFGS:   32 14:06:44       10.495335        0.9247
BFGS:   33 14:06:44       10.455566        1.3368
BFGS:   34 14:06:44       11.138043        4.0237
BFGS:   35 14:06:44       10.734698        3.7269
BFGS:   36 14:06:44       10.528278        3.7812
BFGS:   37 14:06:44       10.340476        3.4774
BFGS:   38 14:06:44       10.183097        2.6701
BFGS:   39 14:06:44       10.168530        1.2393
BFGS:   40 14:06:44       10.079878        0.8266
BFGS:   41 14:06:44       10.071043        1.4005
BFGS:   42 14:06:44       10.030463        0.9328
BFGS:   43 14:06:44        9.994388        0.8255
BFGS:   44 14:06:44        9.939437        0.9584


BFGS:   75 14:06:45        8.166154        0.3274
BFGS:   76 14:06:45        8.146463        0.4316
BFGS:   77 14:06:45        8.146860        0.6526
BFGS:   78 14:06:45        8.130163        0.6790
BFGS:   79 14:06:45        8.094235        0.3780
BFGS:   80 14:06:45        8.066018        0.3673
BFGS:   81 14:06:45        8.087692        1.0897
BFGS:   82 14:06:45        8.025182        0.2806
BFGS:   83 14:06:45        8.013415        0.3273
BFGS:   84 14:06:45        7.992597        0.2522
BFGS:   85 14:06:45        7.981686        0.2085
BFGS:   86 14:06:45        7.970387        0.1976
BFGS:   87 14:06:45        7.963910        0.2029
BFGS:   88 14:06:45        7.954191        0.2425
BFGS:   89 14:06:45        7.945107        0.2172
BFGS:   90 14:06:45        7.938171        0.1486
BFGS:   91 14:06:45        7.933507        0.1676
BFGS:   92 14:06:45        7.930553        0.1445
BFGS:   93 14:06:45        7.927535        0.1113
BFGS:   94 14:06:45        7.924326        0.0881


BFGS:   61 14:07:29        7.270241        0.3670
BFGS:   62 14:07:29        7.254655        0.3759
BFGS:   63 14:07:29        7.241558        0.2656
BFGS:   64 14:07:29        7.227960        0.2998
BFGS:   65 14:07:29        7.209118        0.5099
BFGS:   66 14:07:29        7.195537        0.5144
BFGS:   67 14:07:29        7.178919        0.3946
BFGS:   68 14:07:29        7.170798        0.2866
BFGS:   69 14:07:29        7.160844        0.4422
BFGS:   70 14:07:29        7.142180        0.2482
BFGS:   71 14:07:29        7.121527        0.2542
BFGS:   72 14:07:29        7.105170        0.3514
BFGS:   73 14:07:29        7.091548        0.4381
BFGS:   74 14:07:29        7.070462        0.5117
BFGS:   75 14:07:29        7.044636        0.4792
BFGS:   76 14:07:29        7.010077        0.4557
BFGS:   77 14:07:29        6.966433        0.4700
BFGS:   78 14:07:29        6.951992        0.4436
BFGS:   79 14:07:29        6.930829        0.3683
BFGS:   80 14:07:29        6.921771        0.3010


BFGS:    1 14:08:16        7.376997        0.7749
BFGS:    2 14:08:16        7.342133        0.4303
BFGS:    3 14:08:16        7.338380        0.2899
BFGS:    4 14:08:16        7.331894        0.1516
BFGS:    5 14:08:16        7.330171        0.1089
BFGS:    6 14:08:16        7.328450        0.1132
BFGS:    7 14:08:16        7.325939        0.1394
BFGS:    8 14:08:16        7.323447        0.1346
BFGS:    9 14:08:16        7.321817        0.0999
BFGS:   10 14:08:16        7.320821        0.0724
BFGS:   11 14:08:16        7.319815        0.0707
BFGS:   12 14:08:16        7.318692        0.0664
BFGS:   13 14:08:16        7.317830        0.0619
BFGS:   14 14:08:16        7.317255        0.0579
BFGS:   15 14:08:16        7.316761        0.0547
BFGS:   16 14:08:16        7.316183        0.0476
      Step     Time          Energy         fmax
BFGS:    0 14:08:16        7.590261        0.0499
      Step     Time          Energy         fmax
BFGS:    0 14:08:16        7.590261        0.0499
  

<Popen: returncode: None args: ['/home/jon/Documents/GrandCanonicalGA/ase_en...>